In [1]:
import pandas as pd
import numpy as np
import requests
import json
from pandas_ods_reader import read_ods

In [2]:
# retrieve data from api on the web
url = 'http://chargepoints.dft.gov.uk/api/retrieve/registry/format/csv'
chargepoint_df = pd.read_csv(url)

# COMBINED WITH DATA FROM OCM (OPEN CHARGE MAP)

In [ ]:
# get shape of data
chargepoint_df.head()

,chargeDeviceID,reference,name,latitude,longitude,subBuildingName,buildingName,buildingNumber,thoroughfare,street,...,connector8Type,connector8RatedOutputKW,connector8OutputCurrent,connector8RatedVoltage,connector8ChargeMethod,connector8ChargeMode,connector8TetheredCable,connector8Status,connector8Description,connector8Validated
0,c911241d00294e8bb714eee2e83fa475,PP-12289,Alex F Noble & Son,55.875053,-3.173333,NaN,NaN,NaN,NaN,Swinton Place,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,fa6c94460e902005a0b660266190c8ba,PP-12295,Ancaster Nissan Dealership,51.411173,-0.055369,NaN,NaN,61,Croydon Road,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,eb1848290d5a7de9c9ccabc67fefa211,PP-12290,Beadles Nissan Ltd,51.451127,0.050619,NaN,NaN,43-53,Eltham High Street,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,91e50fe1e39af2869d3336eaaeebdb43,PP-12292,Benfield Motors,54.978947,-1.599306,NaN,NaN,176,Portland Road,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,65b1e92c585fd4c2159d5f33b5030ff2,PP-12198,Circus Road,51.533633,-0.172353,NaN,NaN,NaN,Circus Road,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
chargepoint_df.shape

(31702, 158)

In [ ]:



# retreive comments from openchargemap api
url = 'https://api.openchargemap.io/v3/poi/?output=json&countrycode=GB&maxresults=100000&compact=true&verbose=false&key=1e2b0b1e-5b1e-4b3a-8b9a-0b8b9a3a2b1e&includecomments=true'
response = requests.get(url)
data = response.json()

# convert json to dataframe
ocm_comments_df = pd.DataFrame(data)
ocm_comments_df.to_csv('data/ocm_comments.csv')



In [ ]:
ocm_comments_df.shape

(25344, 23)

In [ ]:
ocm_comments_df.head()
# unnest the column AddressInfo and UserComments
ocm_comments_df = pd.concat([ocm_comments_df.drop(['AddressInfo'], axis=1), ocm_comments_df['AddressInfo'].apply(pd.Series)], axis=1)

# COMBINED WITH CHARGEPOINT DATA

In [ ]:
# join the two dataframes using latitude and longitude
chargepoint_df = chargepoint_df.merge(ocm_comments_df, how='left', left_on=['latitude', 'longitude'], right_on=['Latitude', 'Longitude'])

# COMBINED DATA (OCM + CHARGEPOINT)

In [ ]:

# save the data to csv
chargepoint_df.to_csv('data/chargepoint.csv')

chargepoint_df.head()


,chargeDeviceID,reference,name,latitude,longitude,subBuildingName,buildingName,buildingNumber,thoroughfare,street,...,Postcode,CountryID,Latitude,Longitude,DistanceUnit,AccessComments,RelatedURL,ContactTelephone1,ContactTelephone2,ContactEmail
0,c911241d00294e8bb714eee2e83fa475,PP-12289,Alex F Noble & Son,55.875053,-3.173333,NaN,NaN,NaN,NaN,Swinton Place,...,EH20 9FB,1.0,55.875053,-3.173333,0.0,Charge Points Located in Customer Parking Area...,http://afnoble.nissan.co.uk/,0131 440 5353,NaN,NaN
1,fa6c94460e902005a0b660266190c8ba,PP-12295,Ancaster Nissan Dealership,51.411173,-0.055369,NaN,NaN,61,Croydon Road,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,eb1848290d5a7de9c9ccabc67fefa211,PP-12290,Beadles Nissan Ltd,51.451127,0.050619,NaN,NaN,43-53,Eltham High Street,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,91e50fe1e39af2869d3336eaaeebdb43,PP-12292,Benfield Motors,54.978947,-1.599306,NaN,NaN,176,Portland Road,NaN,...,NE2 1AR,1.0,54.978947,-1.599306,0.0,NaN,NaN,020 7247 4114,NaN,NaN
4,65b1e92c585fd4c2159d5f33b5030ff2,PP-12198,Circus Road,51.533633,-0.172353,NaN,NaN,NaN,Circus Road,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
# get electricity data with only sheet name "2.2.4" and skipping first 12 rows
electricity_df = pd.read_excel('data/table_224.xlsx', sheet_name='2.2.4', skiprows=12)
# rename the columns
electricity_df.columns = ['Year', 'Region', 'PES area', '1', '2', '3', '4', '5', '6', 'Average variable unit price (£/kWh)', '7']
# drop columns 1-7
electricity_df = electricity_df.drop(['1', '2', '3', '4', '5', '6', '7'], axis=1)
# drop Region column
electricity_df = electricity_df.drop(['Region'], axis=1)
electricity_df.head()

# COMBINED WITH FUEL PRICES

,Year,PES area,Average variable unit price (£/kWh)
0,2010,Northern Scotland,0.118306
1,2010,Northern Ireland,0.147227
2,2010,West Midlands,0.113377
3,2010,South East,0.109783
4,2010,South Wales,0.122350


In [ ]:
# show unique values in PES area
electricity_df['PES area'].unique()

array(['Northern Scotland', 'Northern Ireland', 'West Midlands',
       'South East', 'South Wales', 'Southern Scotland', 'Eastern',
       'Yorkshire', 'Merseyside & North Wales', 'London', 'North West',
       'North East', 'East Midlands', 'South West', 'Southern',
       'United Kingdom', 'North Scotland', 'South Scotland'], dtype=object)

In [ ]:
# replace PES area values with the correct names

electricity_df['PES area'] = electricity_df['PES area'].replace(['Northern Scotland'], 'North Scotland')
electricity_df['PES area'] = electricity_df['PES area'].replace(['Southern Scotland'], 'South Scotland')
# check if the values are replaced
electricity_df['PES area'].unique()

array(['North Scotland', 'Northern Ireland', 'West Midlands',
       'South East', 'South Wales', 'South Scotland', 'Eastern',
       'Yorkshire', 'Merseyside & North Wales', 'London', 'North West',
       'North East', 'East Midlands', 'South West', 'Southern',
       'United Kingdom'], dtype=object)

In [ ]:
# gdhi data
gdhi_df = pd.read_excel("data/regionalgrossdisposablehouseholdincomeallitlregions.xls", sheet_name="Table 3", skiprows=1)
# melt the dataframe
gdhi_df = pd.melt(gdhi_df, id_vars=['Region name', 'ITL level','ITL code'], var_name='Year', value_vars=['1997', '1998', 
                                                                                                         '1999', '2000', '2001', '2002', 
                                                                                                         '2003','2004', '2005', '2006', 
                                                                                                         '2007', '2008', '2009', '2010',
                                                                                                         '2011', '2012', '2013', '2014', 
                                                                                                         '2015', '2016', '2017','2018', 
                                                                                                         '2019', '2020'])
# rename the column value to gdhi
gdhi_df = gdhi_df.rename(columns={'value': 'gdhi'})
gdhi_df.head()

# COMBINED WITH POPULATION DATA

,Region name,ITL level,ITL code,Year,gdhi
0,United Kingdom,UK,UK,1997,10757
1,England,Other,TLB,1997,11016
2,North East,ITL1,TLC,1997,9253
3,Tees Valley and Durham,ITL2,TLC1,1997,9200
4,Hartlepool and Stockton-on-Tees,ITL3,TLC11,1997,9264


In [ ]:
# population data
population_df = pd.read_csv('data/ukdetailedtimeseries2001to2020/MYEB1_detailed_population_estimates_series_UK_(2020_geog21).csv')
# remove word "population" from column names
population_df.columns = population_df.columns.str.replace('population_', '')

# melt the dataframe
population_df = pd.melt(population_df, id_vars=['ladcode21', 'laname21','country','sex','age'], var_name='Year', 
                        value_vars=['2001', '2002', '2003', '2004', '2005', '2006', '2007','2008', '2009', '2010', '2011', '2012', 
                                    '2013','2014', '2015', '2016', '2017', '2018', '2019', '2020'])
# rename the column value to population
population_df = population_df.rename(columns={'value': 'population'})

# remove ladcode21 column
population_df = population_df.drop(['ladcode21','country'], axis=1)

# COMBINED WITH GDHI DATA



In [ ]:
# correct the values in laname21 column to match the values in gdhi data "Region name" columns
# change Barking and Dagenham to "Barking & Dagenham and Havering"
population_df['laname21'] = population_df['laname21'].replace(['Barking and Dagenham', 'Havering'], 'Barking & Dagenham and Havering')
# change "Angus" to "Angus and Dundee City"
population_df['laname21'] = population_df['laname21'].replace(['Angus', 'Dundee City'], 'Angus and Dundee City')
# change Maldon to Heart of Essex
population_df.loc[population_df['laname21'].str.contains('Maldon', case=False, na=False), 'laname21'] = 'Heart of Essex'
# change the values in laname21 column to match the values in gdhi data "Region name" columns
# change "Hartlepool" to "Durham CC"
population_df['laname21'] = population_df['laname21'].replace(['Hartlepool'], 'Hartlepool and Stockton-on-Tees')
# change "Aberdeen City" to "Aberdeen City and Aberdeenshire"
population_df['laname21'] = population_df['laname21'].replace(['Aberdeen City'], 'Aberdeen City and Aberdeenshire')
# change "Aberdeenshire" to "Aberdeen City and Aberdeenshire"
population_df['laname21'] = population_df['laname21'].replace(['Aberdeenshire'], 'Aberdeen City and Aberdeenshire')
# change "Adur" to "West Sussex (South West)"
population_df['laname21'] = population_df['laname21'].replace(['Worthing', 'Adur', 'Arun', 'Chichester', 'Horsham', 'Mid Sussex'], 'West Sussex (South West)')
# change "Allerdale" to "West Cumbria"
population_df['laname21'] = population_df['laname21'].replace(['Allerdale', 'Barrow-in-Furness', 'Carlisle', 'Copeland'], 'West Cumbria')
# change "Allerdale" to "West Cumbria"
population_df['laname21'] = population_df['laname21'].replace(['Eden', 'South Lakeland'], 'East Cumbria')
# change "Argyll and Bute" to "Lochaber, Skye and Lochalsh, Arran and Cumbrae and Argyll and Bute"
population_df['laname21'] = population_df['laname21'].replace(['Argyll and Bute'], 'Lochaber, Skye and Lochalsh, Arran and Cumbrae and Argyll and Bute')
# change Ashfield to "North Nottinghamshire"
population_df['laname21'] = population_df['laname21'].replace(['Ashfield', 'Bassetlaw', 'Bolsover', 'Gedling'], 'North Nottinghamshire')
# change Babergh to "Suffolk"
population_df['laname21'] = population_df['laname21'].replace(['Babergh', 'East Suffolk', 'Ipswich', 'Mid Suffolk', 'West Suffolk'], 'Suffolk')
# change Barnsley to "Barnsley, Doncaster and Rotherham"
population_df['laname21'] = population_df['laname21'].replace(['Barnsley', 'Doncaster', 'Rotherham'], 'Barnsley, Doncaster and Rotherham')
# change Basildon to "Thurrock"
population_df['laname21'] = population_df['laname21'].replace(['Basildon'], 'Thurrock')
# change Basingstoke and Deane to "North Hampshire"
population_df['laname21'] = population_df['laname21'].replace(['Basingstoke and Deane', 'Hart', 'Rushmoor'], 'North Hampshire')
# change Bath and North East Somerset to "Bath and North East Somerset, North Somerset and South Gloucestershire"
population_df['laname21'] = population_df['laname21'].replace(['Bath and North East Somerset', 'North Somerset', 'South Gloucestershire'], 'Bath and North East Somerset, North Somerset and South Gloucestershire')
# change Bexley to "Bexley and Greenwich"
population_df['laname21'] = population_df['laname21'].replace(['Bexley', 'Greenwich'], 'Bexley and Greenwich')
# change Blaby to "Leicestershire CC and Rutland"
population_df['laname21'] = population_df['laname21'].replace(['Blaby', 'Charnwood', 'Harborough', 'Hinckley and Bosworth', 'Melton', 'North West Leicestershire', 'Oadby and Wigston', 'Rutland'], 'Leicestershire CC and Rutland')
# change Blaenau Gwent to "Gwent Valleys"
population_df['laname21'] = population_df['laname21'].replace(['Blaenau Gwent', 'Torfaen'], 'Gwent Valleys')
# change Bolton to "Greater Manchester North West"
population_df['laname21'] = population_df['laname21'].replace(['Bolton', 'Bury'], 'Greater Manchester North West')
# change Boston to "Lincolnshire"
population_df['laname21'] = population_df['laname21'].replace(['Boston', 'North Kesteven', 'West Lindsey', 'South Kesteven', 'South Holland'], 'Lincolnshire')
# change Bracknell Forest to "Berkshire"
population_df['laname21'] = population_df['laname21'].replace(['Bracknell Forest', 'Reading', 'Slough', 'West Berkshire', 'Windsor and Maidenhead', 'Wokingham'], 'Berkshire')
# change Braintree to "Essex Haven Gateway"
population_df['laname21'] = population_df['laname21'].replace(['Braintree', 'Brentwood', 'Castle Point', 'Colchester', 'Chelmsford', 'Rochford'], 'Essex Haven Gateway')
# change Breckland to "Breckland and South Norfolk"
population_df['laname21'] = population_df['laname21'].replace(['Breckland', 'Broadland', 'Great Yarmouth', 'South Norfolk'], 'Breckland and South Norfolk')
# change Bridgend to "Bridgend and Neath Port Talbot"
population_df['laname21'] = population_df['laname21'].replace(['Bridgend', 'Merthyr Tydfil', 'Neath Port Talbot', 'Rhondda Cynon Taf'], 'Bridgend and Neath Port Talbot')
#  change Bromsgrove to "Worcestershire"
population_df['laname21'] = population_df['laname21'].replace(['Bromsgrove', 'Malvern Hills', 'Redditch', 'Worcester', 'Wychavon'], 'Worcestershire')
# change Broxbourne to "Hertfordshire"
population_df['laname21'] = population_df['laname21'].replace(['Watford', 'Stevenage', 'St Albans', 'Three Rivers', 'Welwyn Hatfield', 'North Hertfordshire', 'Broxbourne', 'East Hertfordshire', 'Dacorum', 'Hertsmere'], 'Hertfordshire')
# change Broxtowe to "South and West Derbyshire"
population_df['laname21'] = population_df['laname21'].replace(['Broxtowe', 'Amber Valley', 'Derbyshire Dales', 'Erewash', 'South Derbyshire'], 'South and West Derbyshire')
# change Buckinghamshire to "Buckinghamshire CC"
population_df['laname21'] = population_df['laname21'].replace(['Buckinghamshire'], 'Buckinghamshire CC')
# change Burnley to "East Lancashire"
population_df['laname21'] = population_df['laname21'].replace(['Burnley', 'Hyndburn', 'Pendle', 'Ribble Valley', 'Rossendale'], 'East Lancashire')
# change Caerphilly to "Cardiff and Vale of Glamorgan"
population_df['laname21'] = population_df['laname21'].replace(['Caerphilly', 'Cardiff', 'Vale of Glamorgan'], 'Cardiff and Vale of Glamorgan')
# change Cambridge to Cambridgeshire CC
population_df['laname21'] = population_df['laname21'].replace(['Cambridge', 'East Cambridgeshire', 'Fenland', 'Huntingdonshire', 'South Cambridgeshire'], 'Cambridgeshire CC')
# change Cannock Chase to Staffordshire CC
population_df['laname21'] = population_df['laname21'].replace(['Newcastle-under-Lyme', 'Cannock Chase', 'East Staffordshire', 'Lichfield', 'South Staffordshire', 'Tamworth', 'Staffordshire Moorlands', 'Stafford'], 'Staffordshire CC')
# change Carmarthenshire to South West Wales
population_df['laname21'] = population_df['laname21'].replace(['Carmarthenshire', 'Pembrokeshire'], 'South West Wales')
# change Ceredigion to Central Valleys
population_df.loc[population_df['laname21'].str.contains('Ceredigion', case=False, na=False), 'laname21'] = 'Central Valleys'
# change Cherwell to Oxfordshire
population_df['laname21'] = population_df['laname21'].replace(['Cherwell', 'Oxford', 'South Oxfordshire', 'Vale of White Horse', 'West Oxfordshire'], 'Oxfordshire')
# change Chesterfield to East Derbyshire
population_df['laname21'] = population_df['laname21'].replace(['Chesterfield', 'North East Derbyshire', 'High Peak'], 'East Derbyshire')
# change Chorley to Chorley and West Lancashire
population_df['laname21'] = population_df['laname21'].replace(['Chorley', 'Fylde', 'South Ribble', 'West Lancashire'], 'Chorley and West Lancashire')
# change City of London to Camden and City of London
population_df['laname21'] = population_df['laname21'].replace(['City of London', 'Camden'], 'Camden and City of London')
# change Clackmannanshire to Clackmannanshire and Fife
population_df['laname21'] = population_df['laname21'].replace(['Clackmannanshire', 'Fife'], 'Clackmannanshire and Fife')
# change Conwy to Conwy and Denbighshire
population_df['laname21'] = population_df['laname21'].replace(['Conwy', 'Denbighshire'], 'Conwy and Denbighshire')
# change Cornwall to Cornwall and Isles of Scilly
population_df['laname21'] = population_df['laname21'].replace(['Cornwall', 'Isles of Scilly'], 'Cornwall and Isles of Scilly')
# change County Durham to Durham CC
population_df.loc[population_df['laname21'].str.contains('County Durham', case=False, na=False), 'laname21'] = 'Durham CC'
# change Crawley to West Sussex
population_df.loc[population_df['laname21'].str.contains('Crawley', case=False, na=False), 'laname21'] = 'West Sussex (North East)'
# change East Ayrshire to East Ayrshire and North Ayrshire mainland
population_df['laname21'] = population_df['laname21'].replace(['East Ayrshire', 'North Ayrshire'], 'East Ayrshire and North Ayrshire mainland')
# change East Devon to Devon CC
population_df['laname21'] = population_df['laname21'].replace(['East Devon', 'Exeter', 'Mid Devon', 'North Devon', 'South Hams', 'Teignbridge', 'Torridge', 'West Devon'], 'Devon CC')
# change East Dunbartonshire to East Dunbartonshire, West Dunbartonshire and Helensburgh and Lomond
population_df['laname21'] = population_df['laname21'].replace(['East Dunbartonshire', 'West dunbartonshire', 'West Dunbartonshire'], 'East Dunbartonshire, West Dunbartonshire and Helensburgh and Lomond')
# change East Hampshire to Central Hampshire
population_df['laname21'] = population_df['laname21'].replace(['East Hampshire', 'Eastleigh', 'New Forest'], 'Central Hampshire')
# change East Lothian to East Lothian and Midlothian
population_df['laname21'] = population_df['laname21'].replace(['East Lothian', 'Midlothian'], 'East Lothian and Midlothian')
# change East Renfrewshire to Inverclyde, East Renfrewshire and Renfrewshire
population_df['laname21'] = population_df['laname21'].replace(['East Renfrewshire', 'Inverclyde', 'Renfrewshire'], 'Inverclyde, East Renfrewshire and Renfrewshire')
# change Eastbourne to East Sussex
population_df['laname21'] = population_df['laname21'].replace(['Eastbourne', 'East Sussex'])
# change Elmbridge to West Surrey
population_df['laname21'] = population_df['laname21'].replace(['Elmbridge', 'Epsom and Ewell', 'Guildford', 'Runnymede', 'Spelthorne', 'Woking', 'Waverley', 'Surrey Heath'], 'West Surrey')
# change Fareham to South Hampshire
population_df['laname21'] = population_df['laname21'].replace(['Fareham', 'Gosport', 'Havant'], 'South Hampshire')
# change Flintshire to Flintshire and Wrexham
population_df['laname21'] = population_df['laname21'].replace(['Flintshire', 'Wrexham'], 'Flintshire and Wrexham')
# change Folkestone and Hythe to East Kent
population_df['laname21'] = population_df['laname21'].replace(['Folkestone and Hythe', 'Dover', 'Ashford', 'Canterbury'], 'East Kent')
# change Forest of Dean to Gloucestershire
population_df['laname21'] = population_df['laname21'].replace(['Gloucester', 'Forest of Dean', 'Cotswold', 'Cheltenham', 'Stroud', 'Tewkesbury'], 'Gloucestershire')
# change Gateshead to Tyneside
population_df['laname21'] = population_df['laname21'].replace(['Newcastle upon Tyne', 'Gateshead', 'North Tyneside', 'South Tyneside'], 'Tyneside')
# change Hackney to Hackney and Newham
population_df['laname21'] = population_df['laname21'].replace(['Hackney', 'Newham'], 'Hackney and Newham')
# change Hammersmith and Fulham to Kensington & Chelsea and Hammersmith & Fulham
population_df['laname21'] = population_df['laname21'].replace(['Hammersmith and Fulham', 'Kensington and Chelsea'], 'Kensington & Chelsea and Hammersmith & Fulham')
# change Harlow to Essex Thames Gateway
population_df['laname21'] = population_df['laname21'].replace(['Harlow', 'Epping Forest'], 'Essex Thames Gateway')
# change Harrow to Harrow and Hillingdon
population_df['laname21'] = population_df['laname21'].replace(['Harrow', 'Hillingdon'], 'Harrow and Hillingdon')
# change Haringey to Haringey and Islington
population_df['laname21'] = population_df['laname21'].replace(['Haringey', 'Islington'], 'Haringey and Islington')
# change Hastings to East Sussex CC
population_df['laname21'] = population_df['laname21'].replace(['Hastings', 'Lewes', 'Rother', 'Wealden'], 'East Sussex CC')
# change Highland to Highlands and Islands
population_df.loc[population_df['laname21'].str.contains('Highland', case=False, na=False), 'laname21'] = 'Caithness and Sutherland and Ross and Cromarty'
# change Hounslow to Hounslow and Richmond upon Thames
population_df['laname21'] = population_df['laname21'].replace(['Hounslow', 'Richmond upon Thames'], 'Hounslow and Richmond upon Thames')
# change King's Lynn and West Norfolk to North and West Norfolk
population_df['laname21'] = population_df['laname21'].replace(["King's Lynn and West Norfolk", 'North Norfolk'], 'North and West Norfolk')
# change Kingston upon Thames to Merton, Kingston upon Thames and Sutton
population_df['laname21'] = population_df['laname21'].replace(['Kingston upon Thames', 'Merton', 'Sutton'], 'Merton, Kingston upon Thames and Sutton')
# change Kirklees to Calderdale and Kirklees
population_df['laname21'] = population_df['laname21'].replace(['Kirklees', 'Calderdale'], 'Calderdale and Kirklees')
# change Knowsley to East Merseyside
population_df['laname21'] = population_df['laname21'].replace(['Knowsley', 'St. Helens'], 'East Merseyside')
# change Lancaster to Lancaster and Wyre
population_df['laname21'] = population_df['laname21'].replace(['Lancaster', 'Wyre', 'Wyre Forest'], 'Lancaster and Wyre')
# change Lewisham to Lewisham and Southwark
population_df['laname21'] = population_df['laname21'].replace(['Lewisham', 'Southwark'], 'Lewisham and Southwark')
# change Lincoln to North and North East Lincolnshire
population_df['laname21'] = population_df['laname21'].replace(['Lincoln', 'North East Lincolnshire', 'North Lincolnshire', 'East Lindsey'], 'North and North East Lincolnshire')
# change Maidstone to Medway
population_df['laname21'] = population_df['laname21'].replace('Maidstone', 'Medway')
# change Mansfield to Nottingham
population_df['laname21'] = population_df['laname21'].replace(['Mansfield', 'Newark and Sherwood'], 'Nottingham')
# change Mendip to Somerset
population_df['laname21'] = population_df['laname21'].replace(['Mendip', 'South Somerset', 'Somerset West and Taunton', 'Sedgemoor'], 'Somerset')
# change Mole Valley to East Surrey
population_df['laname21'] = population_df['laname21'].replace(['Mole Valley', 'Reigate and Banstead', 'Tandridge'], 'East Surrey')
# change Monmouthshire to Monmouthshire and Newport
population_df['laname21'] = population_df['laname21'].replace(['Monmouthshire', 'Newport'], 'Monmouthshire and Newport')
# change Moray to Inverness and Nairn and Moray, Badenoch and Strathspey
population_df['laname21'] = population_df['laname21'].replace(['Moray'], 'Inverness and Nairn and Moray, Badenoch and Strathspey')
# change North Warwickshire to Warwickshire
population_df['laname21'] = population_df['laname21'].replace(['Warwick', 'North Warwickshire', 'Nuneaton and Bedworth', 'Rugby', 'Stratford-on-Avon'], 'Warwickshire')
# change Norwich to Norwich and East Norfolk
population_df.loc[population_df['laname21'].str.contains('Norwich', case=False, na=False), 'laname21'] = 'Norwich and East Norfolk'
# change Oldham to Greater Manchester North East
population_df['laname21'] = population_df['laname21'].replace(['Oldham', 'Rochdale', 'Salford'], 'Greater Manchester North East')
# change Perth and Kinross to Perth and Kinross and Stirling
population_df['laname21'] = population_df['laname21'].replace(['Perth and Kinross', 'Stirling'], 'Perth and Kinross and Stirling')
# change preston to Mid Lancashire
population_df.loc[population_df['laname21'].str.contains('Preston', case=False, na=False), 'laname21'] = 'Mid Lancashire'
# change 'Redcar and Cleveland' to South Teesside
population_df['laname21'] = population_df['laname21'].replace(['Redcar and Cleveland'], 'South Teesside')
# change Redcar and Cleveland to North Yorkshire CC
population_df['laname21'] = population_df['laname21'].replace(['Richmondshire', 'Middlesbrough', 'Hambleton', 'Harrogate', 'Ryedale', 'Scarborough', 'Selby', 'Craven'], 'North Yorkshire CC')
# change Redbridge to Redbridge and Waltham Forest
population_df['laname21'] = population_df['laname21'].replace(['Redbridge', 'Waltham Forest'], 'Redbridge and Waltham Forest')
# change Rushcliffe to Nottinghamshire
population_df.loc[population_df['laname21'].str.contains('Rushcliffe', case=False, na=False), 'laname21'] = 'South Nottinghamshire'
# change Shropshire to Shropshire CC
population_df.loc[population_df['laname21'].str.contains('Shropshire', case=False, na=False), 'laname21'] = 'Shropshire CC'
# change Stockport to Greater Manchester South East
population_df['laname21'] = population_df['laname21'].replace(['Stockport', 'Tameside', 'Wigan'], 'Greater Manchester South East')
# change 'Trafford' to Greater Manchester South West
population_df['laname21'] = population_df['laname21'].replace(['Trafford'], 'Greater Manchester South West')
# change Stockton-on-Tees to Hartlepool and Stockton-on-Tees
population_df['laname21'] = population_df['laname21'].replace(['Stockton-on-Tees', 'Hartlepool and Stockton-on-Tees'])
# change Tendring to Essex
population_df['laname21'] = population_df['laname21'].replace(['Tendring', 'Uttlesford'], 'West Essex')
# change Test Valley to Central Hampshire
population_df['laname21'] = population_df['laname21'].replace(['Test Valley'], 'Central Hampshire')
# change Winchester to Portsmouth
population_df.loc[population_df['laname21'].str.contains('Winchester', case=False, na=False), 'laname21'] = 'Portsmouth'
# 'Tonbridge and Malling' to west Kent
population_df['laname21'] = population_df['laname21'].replace(['Tonbridge and Malling', 'Tunbridge Wells', 'Sevenoaks'], 'West Kent')
# change Tunbridge Wells to Kent Thames Gateway
population_df['laname21'] = population_df['laname21'].replace(['Thanet', 'Swale', 'Gravesham', 'Dartford'], 'Kent Thames Gateway')

# group the population data by year, laname21, sex and age and sum the population
population_df = population_df.groupby(['Year', 'laname21', 'sex', 'age'])['population'].sum().reset_index()

# check if population data and gdhi data have the same laname21 and Region name and save as text file
set_pop = set(gdhi_df['Region name'].unique()) - set(population_df['laname21'].unique())


In [ ]:
# merge the dataframes gdhi_df and population_df on laname21 and Year
gdhi_population_df = gdhi_df.merge(population_df, how='right', left_on=['Region name', 'Year'], right_on=['laname21', 'Year'])
gdhi_population_df

# rename Region name to ITL level 3
gdhi_population_df = gdhi_population_df.rename(columns={'Region name': 'ITL level 3'})

# drop laname21 column
gdhi_population_df = gdhi_population_df.drop(['laname21'], axis=1)

# remove all rows with ITL2
gdhi_population_df = gdhi_population_df[gdhi_population_df['ITL level'] == 'ITL3']

# Create a new column ITL level 1 such that it is the first 3 letters of ITL code are TLC then value is North East, 
# if TLD then value is North West, 
# if TLE then value is Yorkshire and The Humber, 
# if TLF then value is East Midlands, 
# if TLG then value is West Midlands, 
# if TLH then value is East of England, 
# if TLI then value is London, 
# if TLJ then value is South East, 
# if TLK then value is South West, 
# if TLM then value is Wales, 
# if TLN then value is Scotland, 
# if TLP then value is Northern Ireland
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL code'].str[:3]
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL level 1'].replace(['TLC'], 'North East')
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL level 1'].replace(['TLD'], 'North West')
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL level 1'].replace(['TLE'], 'Yorkshire and The Humber')
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL level 1'].replace(['TLF'], 'East Midlands')
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL level 1'].replace(['TLG'], 'West Midlands')
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL level 1'].replace(['TLH'], 'East of England')
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL level 1'].replace(['TLI'], 'London')
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL level 1'].replace(['TLJ'], 'South East')
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL level 1'].replace(['TLK'], 'South West')
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL level 1'].replace(['TLL'], 'Wales')
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL level 1'].replace(['TLM'], 'Scotland')
gdhi_population_df['ITL level 1'] = gdhi_population_df['ITL level 1'].replace(['TLN'], 'Northern Ireland')

# create a new column ITL level 2 such that it is the first 4 letters of ITL code are TLC1 then value is Tees Valley and Durham,
# if TLC2 then value is Northumberland and Tyne and Wear,
# if TLD1 then value is Cumbria,
# if TLD3 then value is Greater Manchester,
# if TLD4 then value is Lancashire,
# if TLD6 then value is Cheshire,
# if TLD7 then value is Merseyside,
# if TLE1 then value is East Yorkshire and Northern Lincolnshire,
# if TLE2 then value is North Yorkshire,
# if TLE3 then value is South Yorkshire,
# if TLE4 then value is West Yorkshire,
# if TLF1 then value is Derbyshire and Nottinghamshire,
# if TLF2 then value is Leicestershire, Rutland and Northamptonshire,
# if TLF3 then value is Lincolnshire,
# if TLG1 then value is Herefordshire, Worcestershire and Warwickshire,
# if TLG2 then value is Shropshire and Staffordshire,
# if TLG3 then value is West Midlands,
# if TLH1 then value is East Anglia,
# if TLH2 then value is Bedfordshire and Hertfordshire,
# if TLH3 then value is Essex,
# if TLI3 then value is Inner London - West,
# if TLI4 then value is Inner London - East,
# if TLI5 then value is Outer London - East and North East,
# if TLI6 then value is Outer London - South,
# if TLI7 then value is Outer London - West and North West,
# if TLJ1 then value is Berkshire, Buckinghamshire and Oxfordshire,
# if TLJ2 then value is Surrey, East and West Sussex,
# if TLJ3 then value is Hampshire and Isle of Wight,
# if TLJ4 then value is Kent,
# if TLK1 then value is Gloucestershire, Wiltshire and Bristol/Bath area,
# if TLK2 then value is Dorset and Somerset,
# if TLK3 then value is Cornwall and Isles of Scilly,
# if TLK4 then value is Devon,
# if TLL1 then value is West Wales and The Valleys,
# if TLL2 then value is East Wales,
# if TLM5 then value is North Eastern Scotland,
# if TLM6 then value is Highlands and Islands,
# if TLM7 then value is Eastern Scotland,
# if TLM8 then value is West Central Scotland,
# if TLM9 then value is Southern Scotland,
# if TLN0 then value is Northern Ireland
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL code'].str[:4]
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLC1'], 'Tees Valley and Durham')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLC2'], 'Northumberland and Tyne and Wear')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLD1'], 'Cumbria')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLD3'], 'Greater Manchester')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLD4'], 'Lancashire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLD6'], 'Cheshire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLD7'], 'Merseyside')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLE1'], 'East Yorkshire and Northern Lincolnshire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLE2'], 'North Yorkshire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLE3'], 'South Yorkshire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLE4'], 'West Yorkshire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLF1'], 'Derbyshire and Nottinghamshire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLF2'], 'Leicestershire, Rutland and Northamptonshire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLF3'], 'Lincolnshire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLG1'], 'Herefordshire, Worcestershire and Warwickshire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLG2'], 'Shropshire and Staffordshire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLG3'], 'West Midlands')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLH1'], 'East Anglia')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLH2'], 'Bedfordshire and Hertfordshire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLH3'], 'Essex')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLI3'], 'Inner London - West')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLI4'], 'Inner London - East')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLI5'], 'Outer London - East and North East')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLI6'], 'Outer London - South')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLI7'], 'Outer London - West and North West')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLJ1'], 'Berkshire, Buckinghamshire and Oxfordshire')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLJ2'], 'Surrey, East and West Sussex')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLJ3'], 'Hampshire and Isle of Wight')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLJ4'], 'Kent')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLK1'], 'Gloucestershire, Wiltshire and Bristol/Bath area')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLK2'], 'Dorset and Somerset')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLK3'], 'Cornwall and Isles of Scilly')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLK4'], 'Devon')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLL1'], 'West Wales and The Valleys')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLL2'], 'East Wales')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLM5'], 'North Eastern Scotland')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLM6'], 'Highlands and Islands')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLM7'], 'Eastern Scotland')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLM8'], 'West Central Scotland')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLM9'], 'Southern Scotland')
gdhi_population_df['ITL level 2'] = gdhi_population_df['ITL level 2'].replace(['TLN0'], 'Northern Ireland')


gdhi_population_df.head()

# COMBINED GDHI AND POPULATION DATA

,ITL level 3,ITL level,ITL code,Year,gdhi,sex,age,population,ITL level 1,ITL level 2
0,Aberdeen City and Aberdeenshire,ITL3,TLM50,2001,12622.0,1,0,2267,Scotland,North Eastern Scotland
1,Aberdeen City and Aberdeenshire,ITL3,TLM50,2001,12622.0,1,1,2373,Scotland,North Eastern Scotland
2,Aberdeen City and Aberdeenshire,ITL3,TLM50,2001,12622.0,1,2,2445,Scotland,North Eastern Scotland
3,Aberdeen City and Aberdeenshire,ITL3,TLM50,2001,12622.0,1,3,2481,Scotland,North Eastern Scotland
4,Aberdeen City and Aberdeenshire,ITL3,TLM50,2001,12622.0,1,4,2569,Scotland,North Eastern Scotland


In [ ]:
# show unique values in ITL level 1
gdhi_population_df['ITL level 1'].unique()

array(['Scotland', 'Northern Ireland', 'London',
       'Yorkshire and The Humber', 'South West', 'East of England',
       'South East', 'West Midlands', 'North West', 'Wales', 'North East',
       'East Midlands'], dtype=object)

In [ ]:
path = "data/veh0105.ods"

# load a sheet based on its index (1 based)
sheet_idx = 4
vehicle_fuel_df = read_ods(path, sheet_idx)

# make third row the header
vehicle_fuel_df.columns = vehicle_fuel_df.iloc[3]
# drop first 4 rows
vehicle_fuel_df = vehicle_fuel_df.iloc[4:]
vehicle_fuel_df.head()

# renumber index
vehicle_fuel_df = vehicle_fuel_df.reset_index(drop=True)

# melt the dataframe
vehicle_fuel_df = pd.melt(vehicle_fuel_df, id_vars=['Units', 'BodyType', 'Fuel [note 2]', 'Keepership [note 3]', 'ONS Sort [note 6]', 
                                                    'ONS Code [note 6]', 'ONS Geography [note 6]'], var_name='Year', 
                                                    value_vars = ['2022 Q4', '2022 Q3', '2022 Q2', '2022 Q1', '2021 Q4', '2021 Q3', 
                                                                  '2021 Q2', '2021 Q1', '2020 Q4', '2020 Q3', '2020 Q2', '2020 Q1', 
                                                                  '2019 Q4', '2019 Q3', '2019 Q2', '2019 Q1', '2018 Q4', '2018 Q3', 
                                                                  '2018 Q2', '2018 Q1', '2017 Q4', '2017 Q3', '2017 Q2', '2017 Q1', 
                                                                  '2016 Q4', '2016 Q3', '2016 Q2', '2016 Q1', '2015 Q4', '2015 Q3', 
                                                                  '2015 Q2', '2015 Q1', '2014 Q4', '2014 Q3', '2014 Q2', '2014 Q1', 
                                                                  '2013 Q4', '2013 Q3', '2013 Q2', '2013 Q1', '2012 Q4', '2012 Q3', 
                                                                  '2012 Q2', '2012 Q1', '2011 Q4', '2011 Q3', '2011 Q2', '2011 Q1', 
                                                                  '2010 Q4', '2010 Q3', '2010 Q2', '2010 Q1', '2009 Q4'])
# rename the column value to number of vehicles
vehicle_fuel_df = vehicle_fuel_df.rename(columns={'value': 'Number of vehicles'})
# rename the columns to remove the square brackets
vehicle_fuel_df = vehicle_fuel_df.rename(columns={'Fuel [note 2]': 'Fuel', 'Keepership [note 3]': 'Keepership',
                                                    'ONS Sort [note 6]': 'ONS Sort', 'ONS Code [note 6]': 'ONS Code',
                                                    'ONS Geography [note 6]': 'ONS Geography'})
vehicle_fuel_df.head()

# COMBINED WITH ELECTRIC VEHICLE DATA

,Units,BodyType,Fuel,Keepership,ONS Sort,ONS Code,ONS Geography,Year,Number of vehicles
0,Thousands,Buses and coaches,Diesel,Company,1.0,K02000001,United Kingdom,2022 Q4,110.929
1,Thousands,Buses and coaches,Diesel,Company,2.0,K03000001,Great Britain,2022 Q4,106.768
2,Thousands,Buses and coaches,Diesel,Company,3.0,E92000001,England,2022 Q4,90.01
3,Thousands,Buses and coaches,Diesel,Company,4.0,E12000001,North East,2022 Q4,4.103
4,Thousands,Buses and coaches,Diesel,Company,5.0,E06000047,County Durham,2022 Q4,0.608


In [ ]:
path = "data/veh0142.ods"

# load a sheet based on its index (1 based)
sheet_idx = 4
vehicle_electricity_df = read_ods(path, sheet_idx)

# make third row the header
vehicle_electricity_df.columns = vehicle_electricity_df.iloc[3]
# drop first 4 rows
vehicle_electricity_df = vehicle_electricity_df.iloc[4:]
vehicle_electricity_df.head()

# renumber index
vehicle_electricity_df = vehicle_electricity_df.reset_index(drop=True)
# melt the dataframe
vehicle_electricity_df = pd.melt(vehicle_electricity_df, id_vars=['Units', 'BodyType', 'Fuel', 'Keepership [note 3]', 
                                                                  'ONS Sort [note 6]', 'ONS Code [note 6]', 'ONS Geography [note 6]'], 
                                                                  var_name='Year', 
                                                                  value_vars = ['2022 Q3', '2022 Q2', '2022 Q1', '2021 Q4', '2021 Q3', 
                                                                                '2021 Q2', '2021 Q1', '2020 Q4', '2020 Q3', '2020 Q2',
                                                                                '2020 Q1', '2019 Q4', '2019 Q3', '2019 Q2', '2019 Q1', 
                                                                                '2018 Q4', '2018 Q3', '2018 Q2', '2018 Q1', '2017 Q4', 
                                                                                '2017 Q3', '2017 Q2', '2017 Q1', '2016 Q4', '2016 Q3', 
                                                                                '2016 Q2', '2016 Q1', '2015 Q4', '2015 Q3', '2015 Q2', 
                                                                                '2015 Q1', '2014 Q4', '2014 Q3', '2014 Q2', '2014 Q1', 
                                                                                '2013 Q4', '2013 Q3', '2013 Q2', '2013 Q1', '2012 Q4', 
                                                                                '2012 Q3', '2012 Q2', '2012 Q1', '2011 Q4', '2011 Q3', 
                                                                                '2011 Q2', '2011 Q1', '2010 Q4', '2010 Q3', '2010 Q2', 
                                                                                '2010 Q1', '2009 Q4'])
# rename the column value to "Number of vehicles"
vehicle_electricity_df = vehicle_electricity_df.rename(columns={'value': 'Number of vehicles'})
# rename the columns to remove the square brackets
vehicle_electricity_df = vehicle_electricity_df.rename(columns={'Keepership [note 3]': 'Keepership',
                                                                'ONS Sort [note 6]': 'ONS Sort',
                                                                'ONS Code [note 6]': 'ONS Code',
                                                                'ONS Geography [note 6]': 'ONS Geography'})
vehicle_electricity_df.head()

# COMBINED WITH OTHER VEHICLE DATA

,Units,BodyType,Fuel,Keepership,ONS Sort,ONS Code,ONS Geography,Year,Number of vehicles
0,Number,Buses and coaches,Battery electric,Company,1.0,K02000001,United Kingdom,2022 Q3,1808.0
1,Number,Buses and coaches,Battery electric,Company,2.0,K03000001,Great Britain,2022 Q3,1702.0
2,Number,Buses and coaches,Battery electric,Company,3.0,E92000001,England,2022 Q3,1343.0
3,Number,Buses and coaches,Battery electric,Company,4.0,E12000001,North East,2022 Q3,21.0
4,Number,Buses and coaches,Battery electric,Company,5.0,E06000047,County Durham,2022 Q3,[c]


In [ ]:
# combine the two vehicle dataframes
vehicle_df = pd.concat([vehicle_fuel_df, vehicle_electricity_df])
# remove last 3 characters from year column
vehicle_df['Year'] = vehicle_df['Year'].str[:-3]
# convert year column to integer
vehicle_df['Year'] = vehicle_df['Year'].astype(int)
# replace any text in number of vehicles column with 0
vehicle_df['Number of vehicles'] = vehicle_df['Number of vehicles'].replace('[x]', 0)
vehicle_df['Number of vehicles'] = vehicle_df['Number of vehicles'].replace('[c]', 0)
# if number of vehicles is NaN or not a number, set to 0
vehicle_df['Number of vehicles'] = vehicle_df['Number of vehicles'].fillna(0)
# convert number of vehicles column to integer
vehicle_df['Number of vehicles'] = vehicle_df['Number of vehicles'].astype(int)
# aggregate the number of vehicles by year
vehicle_df = vehicle_df.groupby(['Year', 'Units', 'BodyType', 'Fuel', 'Keepership', 'ONS Sort', 'ONS Code', 'ONS Geography']).agg({'Number of vehicles': 'sum'}).reset_index()
vehicle_df

# COMBINED DATAFRAME (VEHICLE AND EV)

,Year,Units,BodyType,Fuel,Keepership,ONS Sort,ONS Code,ONS Geography,Number of vehicles
0,2009,Number,Buses and coaches,Battery electric,Company,1.0,K02000001,United Kingdom,0
1,2009,Number,Buses and coaches,Battery electric,Company,2.0,K03000001,Great Britain,38
2,2009,Number,Buses and coaches,Battery electric,Company,3.0,E92000001,England,36
3,2009,Number,Buses and coaches,Battery electric,Company,4.0,E12000001,North East,0
4,2009,Number,Buses and coaches,Battery electric,Company,5.0,E06000047,County Durham,0
...,...,...,...,...,...,...,...,...,...
1299769,2022,Thousands,Total,Total,Total,462.0,N09000010,"Newry, Mourne and Down",494
1299770,2022,Thousands,Total,Total,Total,463.0,[z],Local Authority unknown within Northern Ire...,14
1299771,2022,Thousands,Total,Total,Total,464.0,[z],"Vehicle under disposal, previously GB",2914
1299772,2022,Thousands,Total,Total,Total,465.0,[z],"Vehicle under disposal, previously NI",56


In [ ]:
# load a sheet with car models bought and quarter they were purchased
car_models_df = pd.read_csv('data/vehicle-licensing-statistics-2022-data-files/df_VEH0120_GB.csv')
# melt the dataframe
car_models_df = pd.melt(car_models_df, id_vars=['BodyType', 'Make', 'GenModel', 'Model', 'Fuel'], var_name='Year', 
                        value_vars = ['2022Q3', '2022Q2', '2022Q1', '2021Q4', '2021Q3', '2021Q2', '2021Q1', '2020Q4', '2020Q3',
                                        '2020Q2', '2020Q1', '2019Q4', '2019Q3', '2019Q2', '2019Q1', '2018Q4', '2018Q3', '2018Q2',
                                        '2018Q1', '2017Q4', '2017Q3', '2017Q2', '2017Q1', '2016Q4', '2016Q3', '2016Q2', '2016Q1',
                                        '2015Q4', '2015Q3', '2015Q2', '2015Q1', '2014Q4', '2014Q3', '2014Q2', '2014Q1', '2013Q4',
                                        '2013Q3', '2013Q2', '2013Q1', '2012Q4', '2012Q3', '2012Q2', '2012Q1', '2011Q4', '2011Q3',
                                        '2011Q2', '2011Q1', '2010Q4', '2010Q3', '2010Q2', '2010Q1', '2009Q4', '2009Q3', '2009Q2',
                                        '2009Q1', '2008Q4', '2008Q3', '2007Q4', '2006Q4', '2005Q4', '2004Q4', '2003Q4', '2002Q4', 
                                        '2001Q4', '2000Q4', '1999Q4', '1998Q4', '1997Q4', '1996Q4', '1995Q4', '1994Q4'])
# aggregate the number of cars bought by year, make, genmodel, model, fuel and year
# fill the NaN values with 0
car_models_df['value'] = car_models_df['value'].fillna(0)
car_models_df = car_models_df.groupby(['Year', 'BodyType', 'Make', 'GenModel', 'Model', 'Fuel']).agg({'value': 'sum'}).reset_index()
# rename the column value to "Number of vehicles"
car_models_df = car_models_df.rename(columns={'value': 'Number of vehicles'})
car_models_df.head()

,Year,BodyType,Make,GenModel,Model,Fuel,Number of vehicles
0,1994Q4,Buses and coaches,AIXAM,AIXAM MODEL MISSING,MISSING,Diesel,0
1,1994Q4,Buses and coaches,ALBION,ALBION MODEL MISSING,MISSING,Diesel,26
2,1994Q4,Buses and coaches,ALBION,ALBION MODEL MISSING,MISSING,Petrol,7
3,1994Q4,Buses and coaches,ALEXANDER DENNIS,ALEXANDER DENNIS MODEL MISSING,MISSING,Battery electric,0
4,1994Q4,Buses and coaches,ALEXANDER DENNIS,ALEXANDER DENNIS MODEL MISSING,MISSING,Diesel,0


In [ ]:
# get data with fuel prices
fuel_prices_df = pd.read_excel('data/Weekly_Fuel_Prices_120623.xlsx', sheet_name='All years', skiprows=7)
# keep columns 1, 2, 7
fuel_prices_df = fuel_prices_df.iloc[:, [0, 1, 6]]
# aggregate by quarter and get mean
fuel_prices_df = fuel_prices_df.groupby(fuel_prices_df['Date'].dt.to_period("Q")).mean()
# aggregate by year and get mean
fuel_prices_df = fuel_prices_df.groupby(fuel_prices_df.index.year).mean()
# reset index
fuel_prices_df = fuel_prices_df.reset_index()
# rename column Date to Year
fuel_prices_df = fuel_prices_df.rename(columns={'Date': 'Year'})
fuel_prices_df

# COMBINED WITH ELECTRICITY PRICES DATA

,Year,ULSP: Pump price (p/litre),ULSD: Pump price (p/litre)
0,2003,75.330148,77.139578
1,2004,80.254930,81.894003
2,2005,86.796860,90.820266
3,2006,91.395871,95.179094
4,2007,94.384374,96.984666
5,2008,107.001178,117.556702
6,2009,99.593804,104.146614
7,2010,116.904146,119.234615
8,2011,133.412774,138.805382
9,2012,135.761462,142.171851


In [ ]:
# combine electric and fuel prices dataframes by year and left join fuel to electric
fuel_electric_prices_df = pd.merge(electricity_df, fuel_prices_df, on='Year', how='left')
fuel_electric_prices_df

# combine North Scotland and South Scotland into Scotland and rename
fuel_electric_prices_df.loc[fuel_electric_prices_df['PES area'].str.contains('North Scotland', case=False, na=False), 'PES area'] = 'Scotland'
fuel_electric_prices_df.loc[fuel_electric_prices_df['PES area'].str.contains('South Scotland', case=False, na=False), 'PES area'] = 'Scotland'
# combine Merseyside & North Wales and South Wales into Wales and rename
fuel_electric_prices_df.loc[fuel_electric_prices_df['PES area'].str.contains('Merseyside & North Wales', case=False, na=False), 'PES area'] = 'Wales'
fuel_electric_prices_df.loc[fuel_electric_prices_df['PES area'].str.contains('South Wales', case=False, na=False), 'PES area'] = 'Wales'
# replace Eastern with East of England
fuel_electric_prices_df['PES area'] = fuel_electric_prices_df['PES area'].replace('Eastern', 'East of England')
# replace Yorkshire with Yorkshire and The Humber
fuel_electric_prices_df['PES area'] = fuel_electric_prices_df['PES area'].replace('Yorkshire', 'Yorkshire and The Humber')
# drop rows with PES area as "United Kingdom" and "Southern"
fuel_electric_prices_df = fuel_electric_prices_df[fuel_electric_prices_df['PES area'] != 'United Kingdom']
fuel_electric_prices_df = fuel_electric_prices_df[fuel_electric_prices_df['PES area'] != 'Southern']

# aggregate all the data by year and PES area and get the mean
fuel_electric_prices_df = fuel_electric_prices_df.groupby(['Year', 'PES area']).mean().reset_index()

# show unique values in PES Area column
fuel_electric_prices_df['PES area'].unique()
# COMBINED DATAFRAME (FUEL + ELECTRICITY PRICES)

array(['East Midlands', 'East of England', 'London', 'North East',
       'North West', 'Northern Ireland', 'Scotland', 'South East',
       'South West', 'Wales', 'West Midlands', 'Yorkshire and The Humber'],
      dtype=object)

In [ ]:
# set Year to integer
fuel_electric_prices_df['Year'] = fuel_electric_prices_df['Year'].astype(int)
gdhi_population_df['Year'] = gdhi_population_df['Year'].astype(int)
# add fuel_electric_prices_df to gdhi_population_df
gdhi_population_fuel_electric_df = gdhi_population_df.merge(fuel_electric_prices_df, how='inner', left_on=['Year', 'ITL level 1'], right_on=['Year', 'PES area'])
gdhi_population_fuel_electric_df
# drop PES area column

# COMBINED DATAFRAME (GDHI + POPULATION + FUEL + ELECTRICITY PRICES)

,ITL level 3,ITL level,ITL code,Year,gdhi,sex,age,population,ITL level 1,ITL level 2,PES area,Average variable unit price (£/kWh),ULSP: Pump price (p/litre),ULSD: Pump price (p/litre)
0,Aberdeen City and Aberdeenshire,ITL3,TLM50,2010,18736.0,1,0,2801,Scotland,North Eastern Scotland,Scotland,0.117261,116.904146,119.234615
1,Aberdeen City and Aberdeenshire,ITL3,TLM50,2010,18736.0,1,1,2762,Scotland,North Eastern Scotland,Scotland,0.117261,116.904146,119.234615
2,Aberdeen City and Aberdeenshire,ITL3,TLM50,2010,18736.0,1,2,2741,Scotland,North Eastern Scotland,Scotland,0.117261,116.904146,119.234615
3,Aberdeen City and Aberdeenshire,ITL3,TLM50,2010,18736.0,1,3,2596,Scotland,North Eastern Scotland,Scotland,0.117261,116.904146,119.234615
4,Aberdeen City and Aberdeenshire,ITL3,TLM50,2010,18736.0,1,4,2501,Scotland,North Eastern Scotland,Scotland,0.117261,116.904146,119.234615
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
354349,West Northamptonshire,ITL3,TLF24,2020,22354.0,2,86,768,East Midlands,"Leicestershire, Rutland and Northamptonshire",East Midlands,0.166876,114.092268,119.455858
354350,West Northamptonshire,ITL3,TLF24,2020,22354.0,2,87,621,East Midlands,"Leicestershire, Rutland and Northamptonshire",East Midlands,0.166876,114.092268,119.455858
354351,West Northamptonshire,ITL3,TLF24,2020,22354.0,2,88,586,East Midlands,"Leicestershire, Rutland and Northamptonshire",East Midlands,0.166876,114.092268,119.455858
354352,West Northamptonshire,ITL3,TLF24,2020,22354.0,2,89,451,East Midlands,"Leicestershire, Rutland and Northamptonshire",East Midlands,0.166876,114.092268,119.455858


In [ ]:

# remove all rows with ONS Geography 'Vehicle under disposal, previously GB','Vehicle under disposal, previously NI','Region or county unknown'
vehicle_removed_df = vehicle_df[~vehicle_df['ONS Geography'].isin(['Vehicle under disposal, previously NI','Region or county unknown'])]
# strip whitespace from ONS Geography column from front and back
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].str.strip()

vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Barking and Dagenham', 'Havering'], 'Barking & Dagenham and Havering')
# change "Angus" to "Angus and Dundee City"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Angus', 'Dundee City'], 'Angus and Dundee City')
# change Maldon to Heart of Essex
vehicle_removed_df.loc[vehicle_removed_df['ONS Geography'].str.contains('Maldon', case=False, na=False), 'ONS Geography'] = 'Heart of Essex'
# change the values in laname21 column to match the values in gdhi data "Region name" columns
# change "Hartlepool" to "Durham CC"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Hartlepool'], 'Hartlepool and Stockton-on-Tees')
# change "Aberdeen City" to "Aberdeen City and Aberdeenshire"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Aberdeen City'], 'Aberdeen City and Aberdeenshire')
# change "Aberdeenshire" to "Aberdeen City and Aberdeenshire"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Aberdeenshire'], 'Aberdeen City and Aberdeenshire')
# change "Adur" to "West Sussex (South West)"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Worthing', 'Adur', 'Arun', 'Chichester', 'Horsham', 'Mid Sussex'], 'West Sussex (South West)')
# change "Allerdale" to "West Cumbria"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Allerdale', 'Barrow-in-Furness', 'Carlisle', 'Copeland'], 'West Cumbria')
# change "Allerdale" to "West Cumbria"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Eden', 'South Lakeland'], 'East Cumbria')
# change "Argyll and Bute" to "Lochaber, Skye and Lochalsh, Arran and Cumbrae and Argyll and Bute"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Argyll and Bute'], 'Lochaber, Skye and Lochalsh, Arran and Cumbrae and Argyll and Bute')
# change Ashfield to "North Nottinghamshire"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Ashfield', 'Bassetlaw', 'Bolsover', 'Gedling'], 'North Nottinghamshire')
# change Babergh to "Suffolk"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Babergh', 'East Suffolk', 'Ipswich', 'Mid Suffolk', 'West Suffolk'], 'Suffolk')
# change Barnsley to "Barnsley, Doncaster and Rotherham"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Barnsley', 'Doncaster', 'Rotherham'], 'Barnsley, Doncaster and Rotherham')
# change Basildon to "Thurrock"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Basildon'], 'Thurrock')
# change Basingstoke and Deane to "North Hampshire"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Basingstoke and Deane', 'Hart', 'Rushmoor'], 'North Hampshire')
# change Bath and North East Somerset to "Bath and North East Somerset, North Somerset and South Gloucestershire"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Bath and North East Somerset', 'North Somerset', 'South Gloucestershire'], 'Bath and North East Somerset, North Somerset and South Gloucestershire')
# change Bexley to "Bexley and Greenwich"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Bexley', 'Greenwich'], 'Bexley and Greenwich')
# change Blaby to "Leicestershire CC and Rutland"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Blaby', 'Charnwood', 'Harborough', 'Hinckley and Bosworth', 'Melton', 'North West Leicestershire', 'Oadby and Wigston', 'Rutland'], 'Leicestershire CC and Rutland')
# change Blaenau Gwent to "Gwent Valleys"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Blaenau Gwent', 'Torfaen'], 'Gwent Valleys')
# change Bolton to "Greater Manchester North West"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Bolton', 'Bury'], 'Greater Manchester North West')
# change Boston to "Lincolnshire"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Boston', 'North Kesteven', 'West Lindsey', 'South Kesteven', 'South Holland'], 'Lincolnshire')
# change Bracknell Forest to "Berkshire"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Bracknell Forest', 'Reading', 'Slough', 'West Berkshire', 'Windsor and Maidenhead', 'Wokingham'], 'Berkshire')
# change Braintree to "Essex Haven Gateway"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Braintree', 'Brentwood', 'Castle Point', 'Colchester', 'Chelmsford', 'Rochford'], 'Essex Haven Gateway')
# change Breckland to "Breckland and South Norfolk"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Breckland', 'Broadland', 'Great Yarmouth', 'South Norfolk'], 'Breckland and South Norfolk')
# change Bridgend to "Bridgend and Neath Port Talbot"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Bridgend', 'Merthyr Tydfil', 'Neath Port Talbot', 'Rhondda Cynon Taf'], 'Bridgend and Neath Port Talbot')
#  change Bromsgrove to "Worcestershire"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Bromsgrove', 'Malvern Hills', 'Redditch', 'Worcester', 'Wychavon'], 'Worcestershire')
# change Broxbourne to "Hertfordshire"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Watford', 'Stevenage', 'St Albans', 'Three Rivers', 'Welwyn Hatfield', 'North Hertfordshire', 'Broxbourne', 'East Hertfordshire', 'Dacorum', 'Hertsmere'], 'Hertfordshire')
# change Broxtowe to "South and West Derbyshire"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Broxtowe', 'Amber Valley', 'Derbyshire Dales', 'Erewash', 'South Derbyshire'], 'South and West Derbyshire')
# change Buckinghamshire to "Buckinghamshire CC"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Buckinghamshire'], 'Buckinghamshire CC')
# change Burnley to "East Lancashire"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Burnley', 'Hyndburn', 'Pendle', 'Ribble Valley', 'Rossendale'], 'East Lancashire')
# change Caerphilly to "Cardiff and Vale of Glamorgan"
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Caerphilly', 'Cardiff', 'Vale of Glamorgan'], 'Cardiff and Vale of Glamorgan')
# change Cambridge to Cambridgeshire CC
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Cambridge', 'East Cambridgeshire', 'Fenland', 'Huntingdonshire', 'South Cambridgeshire'], 'Cambridgeshire CC')
# change Cannock Chase to Staffordshire CC
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Newcastle-under-Lyme', 'Cannock Chase', 'East Staffordshire', 'Lichfield', 'South Staffordshire', 'Tamworth', 'Staffordshire Moorlands', 'Stafford'], 'Staffordshire CC')
# change Carmarthenshire to South West Wales
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Carmarthenshire', 'Pembrokeshire'], 'South West Wales')
# change Ceredigion to Central Valleys
vehicle_removed_df.loc[vehicle_removed_df['ONS Geography'].str.contains('Ceredigion', case=False, na=False), 'ONS Geography'] = 'Central Valleys'
# change Cherwell to Oxfordshire
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Cherwell', 'Oxford', 'South Oxfordshire', 'Vale of White Horse', 'West Oxfordshire'], 'Oxfordshire')
# change Chesterfield to East Derbyshire
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Chesterfield', 'North East Derbyshire', 'High Peak'], 'East Derbyshire')
# change Chorley to Chorley and West Lancashire
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Chorley', 'Fylde', 'South Ribble', 'West Lancashire'], 'Chorley and West Lancashire')
# change City of London to Camden and City of London
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['City of London', 'Camden'], 'Camden and City of London')
# change Clackmannanshire to Clackmannanshire and Fife
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Clackmannanshire', 'Fife'], 'Clackmannanshire and Fife')
# change Conwy to Conwy and Denbighshire
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Conwy', 'Denbighshire'], 'Conwy and Denbighshire')
# change Cornwall to Cornwall and Isles of Scilly
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Cornwall', 'Isles of Scilly'], 'Cornwall and Isles of Scilly')
# change County Durham to Durham CC
vehicle_removed_df.loc[vehicle_removed_df['ONS Geography'].str.contains('County Durham', case=False, na=False), 'ONS Geography'] = 'Durham CC'
# change Crawley to West Sussex
vehicle_removed_df.loc[vehicle_removed_df['ONS Geography'].str.contains('Crawley', case=False, na=False), 'ONS Geography'] = 'West Sussex (North East)'
# change East Ayrshire to East Ayrshire and North Ayrshire mainland
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['East Ayrshire', 'North Ayrshire'], 'East Ayrshire and North Ayrshire mainland')
# change East Devon to Devon CC
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['East Devon', 'Exeter', 'Mid Devon', 'North Devon', 'South Hams', 'Teignbridge', 'Torridge', 'West Devon'], 'Devon CC')
# change East Dunbartonshire to East Dunbartonshire, West Dunbartonshire and Helensburgh and Lomond
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['East Dunbartonshire', 'West dunbartonshire', 'West Dunbartonshire'], 'East Dunbartonshire, West Dunbartonshire and Helensburgh and Lomond')
# change East Hampshire to Central Hampshire
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['East Hampshire', 'Eastleigh', 'New Forest'], 'Central Hampshire')
# change East Lothian to East Lothian and Midlothian
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['East Lothian', 'Midlothian'], 'East Lothian and Midlothian')
# change East Renfrewshire to Inverclyde, East Renfrewshire and Renfrewshire
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['East Renfrewshire', 'Inverclyde', 'Renfrewshire'], 'Inverclyde, East Renfrewshire and Renfrewshire')
# change Eastbourne to East Sussex
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Eastbourne', 'East Sussex'])
# change Elmbridge to West Surrey
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Elmbridge', 'Epsom and Ewell', 'Guildford', 'Runnymede', 'Spelthorne', 'Woking', 'Waverley', 'Surrey Heath'], 'West Surrey')
# change Fareham to South Hampshire
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Fareham', 'Gosport', 'Havant'], 'South Hampshire')
# change Flintshire to Flintshire and Wrexham
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Flintshire', 'Wrexham'], 'Flintshire and Wrexham')
# change Folkestone and Hythe to East Kent
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Folkestone and Hythe', 'Dover', 'Ashford', 'Canterbury'], 'East Kent')
# change Forest of Dean to Gloucestershire
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Gloucester', 'Forest of Dean', 'Cotswold', 'Cheltenham', 'Stroud', 'Tewkesbury'], 'Gloucestershire')
# change Gateshead to Tyneside
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Newcastle upon Tyne', 'Gateshead', 'North Tyneside', 'South Tyneside'], 'Tyneside')
# change Hackney to Hackney and Newham
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Hackney', 'Newham'], 'Hackney and Newham')
# change Hammersmith and Fulham to Kensington & Chelsea and Hammersmith & Fulham
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Hammersmith and Fulham', 'Kensington and Chelsea'], 'Kensington & Chelsea and Hammersmith & Fulham')
# change Harlow to Essex Thames Gateway
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Harlow', 'Epping Forest'], 'Essex Thames Gateway')
# change Harrow to Harrow and Hillingdon
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Harrow', 'Hillingdon'], 'Harrow and Hillingdon')
# change Haringey to Haringey and Islington
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Haringey', 'Islington'], 'Haringey and Islington')
# change Hastings to East Sussex CC
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Hastings', 'Lewes', 'Rother', 'Wealden'], 'East Sussex CC')
# change Highland to Highlands and Islands
vehicle_removed_df.loc[vehicle_removed_df['ONS Geography'].str.contains('Highland', case=False, na=False), 'ONS Geography'] = 'Caithness and Sutherland and Ross and Cromarty'
# change Hounslow to Hounslow and Richmond upon Thames
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Hounslow', 'Richmond upon Thames'], 'Hounslow and Richmond upon Thames')
# change King's Lynn and West Norfolk to North and West Norfolk
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(["King's Lynn and West Norfolk", 'North Norfolk'], 'North and West Norfolk')
# change Kingston upon Thames to Merton, Kingston upon Thames and Sutton
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Kingston upon Thames', 'Merton', 'Sutton'], 'Merton, Kingston upon Thames and Sutton')
# change Kirklees to Calderdale and Kirklees
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Kirklees', 'Calderdale'], 'Calderdale and Kirklees')
# change Knowsley to East Merseyside
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Knowsley', 'St. Helens'], 'East Merseyside')
# change Lancaster to Lancaster and Wyre
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Lancaster', 'Wyre', 'Wyre Forest'], 'Lancaster and Wyre')
# change Lewisham to Lewisham and Southwark
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Lewisham', 'Southwark'], 'Lewisham and Southwark')
# change Lincoln to North and North East Lincolnshire
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Lincoln', 'North East Lincolnshire', 'North Lincolnshire', 'East Lindsey'], 'North and North East Lincolnshire')
# change Maidstone to Medway
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace('Maidstone', 'Medway')
# change Mansfield to Nottingham
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Mansfield', 'Newark and Sherwood'], 'Nottingham')
# change Mendip to Somerset
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Mendip', 'South Somerset', 'Somerset West and Taunton', 'Sedgemoor'], 'Somerset')
# change Mole Valley to East Surrey
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Mole Valley', 'Reigate and Banstead', 'Tandridge'], 'East Surrey')
# change Monmouthshire to Monmouthshire and Newport
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Monmouthshire', 'Newport'], 'Monmouthshire and Newport')
# change Moray to Inverness and Nairn and Moray, Badenoch and Strathspey
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Moray'], 'Inverness and Nairn and Moray, Badenoch and Strathspey')
# change North Warwickshire to Warwickshire
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Warwick', 'North Warwickshire', 'Nuneaton and Bedworth', 'Rugby', 'Stratford-on-Avon'], 'Warwickshire')
# change Norwich to Norwich and East Norfolk
vehicle_removed_df.loc[vehicle_removed_df['ONS Geography'].str.contains('Norwich', case=False, na=False), 'ONS Geography'] = 'Norwich and East Norfolk'
# change Oldham to Greater Manchester North East
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Oldham', 'Rochdale', 'Salford'], 'Greater Manchester North East')
# change Perth and Kinross to Perth and Kinross and Stirling
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Perth and Kinross', 'Stirling'], 'Perth and Kinross and Stirling')
# change preston to Mid Lancashire
vehicle_removed_df.loc[vehicle_removed_df['ONS Geography'].str.contains('Preston', case=False, na=False), 'ONS Geography'] = 'Mid Lancashire'
# change 'Redcar and Cleveland' to South Teesside
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Redcar and Cleveland'], 'South Teesside')
# change Redcar and Cleveland to North Yorkshire CC
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Richmondshire', 'Middlesbrough', 'Hambleton', 'Harrogate', 'Ryedale', 'Scarborough', 'Selby', 'Craven'], 'North Yorkshire CC')
# change Redbridge to Redbridge and Waltham Forest
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Redbridge', 'Waltham Forest'], 'Redbridge and Waltham Forest')
# change Rushcliffe to Nottinghamshire
vehicle_removed_df.loc[vehicle_removed_df['ONS Geography'].str.contains('Rushcliffe', case=False, na=False), 'ONS Geography'] = 'South Nottinghamshire'
# change Shropshire to Shropshire CC
vehicle_removed_df.loc[vehicle_removed_df['ONS Geography'].str.contains('Shropshire', case=False, na=False), 'ONS Geography'] = 'Shropshire CC'
# change Stockport to Greater Manchester South East
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Stockport', 'Tameside', 'Wigan'], 'Greater Manchester South East')
# change 'Trafford' to Greater Manchester South West
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Trafford'], 'Greater Manchester South West')
# change Stockton-on-Tees to Hartlepool and Stockton-on-Tees
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Stockton-on-Tees', 'Hartlepool and Stockton-on-Tees'])
# change Tendring to Essex
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Tendring', 'Uttlesford'], 'West Essex')
# change Test Valley to Central Hampshire
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Test Valley'], 'Central Hampshire')
# change Winchester to Portsmouth
vehicle_removed_df.loc[vehicle_removed_df['ONS Geography'].str.contains('Winchester', case=False, na=False), 'ONS Geography'] = 'Portsmouth'
# 'Tonbridge and Malling' to west Kent
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Tonbridge and Malling', 'Tunbridge Wells', 'Sevenoaks'], 'West Kent')
# change Tunbridge Wells to Kent Thames Gateway
vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Thanet', 'Swale', 'Gravesham', 'Dartford'], 'Kent Thames Gateway')




# check which values are in ONS Geography in vehicle_removed_df but not in gdhi_population_fuel_electric_df
remove = set(vehicle_removed_df['ONS Geography'].unique()) - set(gdhi_population_fuel_electric_df['ITL level 3'].unique())
# remove rows with values in remove
vehicle_removed_df = vehicle_removed_df[~vehicle_removed_df['ONS Geography'].isin(remove)]

/var/folders/j9/0qd6cw014pgb7rk0v_28prrh0000gn/T/ipykernel_33343/1121752590.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].str.strip()
/var/folders/j9/0qd6cw014pgb7rk0v_28prrh0000gn/T/ipykernel_33343/1121752590.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  vehicle_removed_df['ONS Geography'] = vehicle_removed_df['ONS Geography'].replace(['Barking and Dagenham', 'Havering'], 'Barking & Dagenham and Havering')
/var/folders/j9

In [ ]:
# get unique values in Fuel column
vehicle_removed_df['Fuel'].unique()
# columns
vehicle_removed_df.columns
# remove ONS Code and ONS Sort columns
vehicle_removed_df = vehicle_removed_df.drop(columns=['ONS Code', 'ONS Sort'])

array(['Battery electric', 'Total', 'Plug-in hybrid electric (diesel)',
       'Plug-in hybrid electric (petrol)', 'Range extended electric',
       'Diesel', 'Hybrid electric (petrol)', 'Other fuels', 'Petrol'],
      dtype=object)

In [ ]:
#Fuel types - ['Battery electric', 'Total', 'Plug-in hybrid electric (diesel)', 'Plug-in hybrid electric (petrol)', 'Range extended electric','Diesel', 'Hybrid electric (petrol)', 'Other fuels', 'Petrol']
# create new dataframe for each fuel type
battery_electric_df = vehicle_removed_df[vehicle_removed_df['Fuel'] == 'Battery electric']
plug_in_hybrid_diesel_df = vehicle_removed_df[vehicle_removed_df['Fuel'] == 'Plug-in hybrid electric (diesel)']
plug_in_hybrid_petrol_df = vehicle_removed_df[vehicle_removed_df['Fuel'] == 'Plug-in hybrid electric (petrol)']
range_extended_electric_df = vehicle_removed_df[vehicle_removed_df['Fuel'] == 'Range extended electric']
diesel_df = vehicle_removed_df[vehicle_removed_df['Fuel'] == 'Diesel']
hybrid_petrol_df = vehicle_removed_df[vehicle_removed_df['Fuel'] == 'Hybrid electric (petrol)']
other_fuels_df = vehicle_removed_df[vehicle_removed_df['Fuel'] == 'Other fuels']
petrol_df = vehicle_removed_df[vehicle_removed_df['Fuel'] == 'Petrol']



In [ ]:
# merge each fuel type dataframe with gdhi_population_fuel_electric_df
battery_electric_gdhi_df = gdhi_population_fuel_electric_df.merge(battery_electric_df, how='inner', left_on=['Year', 'ITL level 3'], right_on=['Year', 'ONS Geography'])
plug_in_hybrid_diesel_gdhi_df = gdhi_population_fuel_electric_df.merge(plug_in_hybrid_diesel_df, how='inner', left_on=['Year', 'ITL level 3'], right_on=['Year', 'ONS Geography'])
plug_in_hybrid_petrol_gdhi_df = gdhi_population_fuel_electric_df.merge(plug_in_hybrid_petrol_df, how='inner', left_on=['Year', 'ITL level 3'], right_on=['Year', 'ONS Geography'])
range_extended_electric_gdhi_df = gdhi_population_fuel_electric_df.merge(range_extended_electric_df, how='inner', left_on=['Year', 'ITL level 3'], right_on=['Year', 'ONS Geography'])
diesel_gdhi_df = gdhi_population_fuel_electric_df.merge(diesel_df, how='inner', left_on=['Year', 'ITL level 3'], right_on=['Year', 'ONS Geography'])
hybrid_petrol_gdhi_df = gdhi_population_fuel_electric_df.merge(hybrid_petrol_df, how='inner', left_on=['Year', 'ITL level 3'], right_on=['Year', 'ONS Geography'])
other_fuels_gdhi_df = gdhi_population_fuel_electric_df.merge(other_fuels_df, how='inner', left_on=['Year', 'ITL level 3'], right_on=['Year', 'ONS Geography'])
petrol_gdhi_df = gdhi_population_fuel_electric_df.merge(petrol_df, how='inner', left_on=['Year', 'ITL level 3'], right_on=['Year', 'ONS Geography'])

# drop ONS Geography column
battery_electric_gdhi_df = battery_electric_gdhi_df.drop(columns=['ONS Geography'])
plug_in_hybrid_diesel_gdhi_df = plug_in_hybrid_diesel_gdhi_df.drop(columns=['ONS Geography'])
plug_in_hybrid_petrol_gdhi_df = plug_in_hybrid_petrol_gdhi_df.drop(columns=['ONS Geography'])
range_extended_electric_gdhi_df = range_extended_electric_gdhi_df.drop(columns=['ONS Geography'])
diesel_gdhi_df = diesel_gdhi_df.drop(columns=['ONS Geography'])
hybrid_petrol_gdhi_df = hybrid_petrol_gdhi_df.drop(columns=['ONS Geography'])
other_fuels_gdhi_df = other_fuels_gdhi_df.drop(columns=['ONS Geography'])
petrol_gdhi_df = petrol_gdhi_df.drop(columns=['ONS Geography'])

# drop nan values
battery_electric_gdhi_df = battery_electric_gdhi_df.dropna()
plug_in_hybrid_diesel_gdhi_df = plug_in_hybrid_diesel_gdhi_df.dropna()
plug_in_hybrid_petrol_gdhi_df = plug_in_hybrid_petrol_gdhi_df.dropna()
range_extended_electric_gdhi_df = range_extended_electric_gdhi_df.dropna()
diesel_gdhi_df = diesel_gdhi_df.dropna()
hybrid_petrol_gdhi_df = hybrid_petrol_gdhi_df.dropna()
other_fuels_gdhi_df = other_fuels_gdhi_df.dropna()
petrol_gdhi_df = petrol_gdhi_df.dropna()





NameError: name 'gdhi_population_fuel_electric_df' is not defined

In [ ]:
# concat columns with electric vehicle dataframes
electric_vehicle_df = pd.concat([battery_electric_gdhi_df, plug_in_hybrid_diesel_gdhi_df, plug_in_hybrid_petrol_gdhi_df, range_extended_electric_gdhi_df, hybrid_petrol_gdhi_df], ignore_index=True)

In [ ]:
electric_vehicle_df.shape

(58886828, 19)

In [ ]:
# chargepoint df save column names in text file
# save column names in text file
with open('chargepoint_columns.txt', 'w') as f:
    for item in chargepoint_df.columns:
        f.write("%s\n" % item)

# show unique values in town column
chargepoint_df['county'].unique()
# compare values in town column with values in ONS Geography column
# get unique values in ONS Geography column
vehicle_removed_df['ONS Geography'].unique()
# get unique values in town column
chargepoint_df['county'].unique()
# get unique values in town column that are not in ONS Geography column


{'Aberdeen City and Aberdeenshire',
 'Angus and Dundee City',
 'Antrim and Newtownabbey',
 'Ards and North Down',
 'Armagh City, Banbridge and Craigavon',
 'Barking & Dagenham and Havering',
 'Barnet',
 'Barnsley, Doncaster and Rotherham',
 'Bath and North East Somerset, North Somerset and South Gloucestershire',
 'Bedford',
 'Belfast',
 'Bexley and Greenwich',
 'Breckland and South Norfolk',
 'Brent',
 'Bridgend and Neath Port Talbot',
 'Bristol, City of',
 'Bromley',
 'Buckinghamshire CC',
 'Caithness and Sutherland and Ross and Cromarty',
 'Calderdale and Kirklees',
 'Cambridgeshire CC',
 'Camden and City of London',
 'Cardiff and Vale of Glamorgan',
 'Causeway Coast and Glens',
 'Central Hampshire',
 'Central Valleys',
 'Chorley and West Lancashire',
 'Clackmannanshire and Fife',
 'Conwy and Denbighshire',
 'Cornwall and Isles of Scilly',
 'Croydon',
 'Derby',
 'Derry City and Strabane',
 'Devon CC',
 'Durham CC',
 'Ealing',
 'East Ayrshire and North Ayrshire mainland',
 'East Cumb

In [ ]:
set(vehicle_removed_df['ONS Geography'].unique()) - set(chargepoint_df['county'].unique())

{'Aberdeen City and Aberdeenshire',
 'Angus and Dundee City',
 'Antrim and Newtownabbey',
 'Ards and North Down',
 'Armagh City, Banbridge and Craigavon',
 'Barking & Dagenham and Havering',
 'Barnet',
 'Barnsley, Doncaster and Rotherham',
 'Bath and North East Somerset, North Somerset and South Gloucestershire',
 'Bedford',
 'Belfast',
 'Bexley and Greenwich',
 'Breckland and South Norfolk',
 'Brent',
 'Bridgend and Neath Port Talbot',
 'Bristol, City of',
 'Bromley',
 'Buckinghamshire CC',
 'Caithness and Sutherland and Ross and Cromarty',
 'Calderdale and Kirklees',
 'Cambridgeshire CC',
 'Camden and City of London',
 'Cardiff and Vale of Glamorgan',
 'Causeway Coast and Glens',
 'Central Hampshire',
 'Central Valleys',
 'Chorley and West Lancashire',
 'Clackmannanshire and Fife',
 'Conwy and Denbighshire',
 'Cornwall and Isles of Scilly',
 'Croydon',
 'Derby',
 'Derry City and Strabane',
 'Devon CC',
 'Durham CC',
 'Ealing',
 'East Ayrshire and North Ayrshire mainland',
 'East Cumb

In [ ]:
set(chargepoint_df['county'].unique()) - set(vehicle_removed_df['ONS Geography'].unique())

{' Lambeth',
 ' Newport',
 ' Somerset',
 '0',
 '07:00',
 '4',
 'Aberdeen City',
 'Aberdeenshire',
 'Aberystwyth',
 'Anglesey',
 'Angus',
 'Angus ',
 'Antrim',
 'Argyl and Bute',
 'Argyll',
 'Argyll & Bute',
 'Argyll and Bute',
 'Armagh',
 'Attleborough',
 'Avon',
 'BERKSHIRE',
 'Barnsley',
 'Basingstoke',
 'Basingstoke and Deane Borough Council',
 'Bath and North East Somerset',
 'Bedfordshire',
 'Beds',
 'Belfast Greater',
 'Bilston',
 'Blaenau Gwent',
 'Bournemouth',
 'Bridgend',
 'Bristol',
 'Buckingham',
 'Buckinghamshire',
 'Bucks',
 'Burton-on-Trent',
 'Bury',
 'Caerphilly',
 'Calne',
 'Cambridgeshire',
 'Cambs',
 'Cardiff',
 'Carmarthenshire',
 'Ceredigion',
 'Cheshire',
 'City of Aberdeen',
 'City of Bristol',
 'City of Cardiff',
 'City of Coventry',
 'City of Glasgow',
 'City of Leicester',
 'City of London',
 'City of Nottingham',
 'City of Westminster',
 'Cleveland',
 'Clwyd',
 'Co Durham',
 'Conwy',
 'Cornwall',
 'County Antrim',
 'County Armagh',
 'County Cork',
 'County D